In [216]:
#import standard libraries 
import numpy as np 
import pandas as pd 
from pandas import Series, DataFrame 
import datetime
import time

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [217]:
ALGORITHM="H2O" #(ALTERNATIVES: XGBOOST, LIGHTGBM, H2O)

**Load Train and Test Data**

In [233]:
train_data = pd.read_csv('./Data/train_data.csv')
test_data = pd.read_csv('./Data/test_data.csv')

**Information on Loaded Data**

In [219]:
print('Train data shape: ',train_data.shape)
print('Train data target distribution: ' , train_data.groupby('income').size())
print('''''')

print('Controlling duplication for train data: ', train_data[train_data.duplicated() == True].shape)
train_data = train_data.drop_duplicates(subset = None, keep = 'first')
print('''''')

print('Train data shape without duplication: ',train_data.shape)
print('Train data target distribution without duplication: ' , train_data.groupby('income').size())

print("------------------------------------------")
print('Test data shape: ',test_data.shape)
print('Test data target distribution: ' , test_data.groupby('income').size())
print('''''')

print('Controlling duplication for test data: ', test_data[test_data.duplicated() == True].shape)
test_data = test_data.drop_duplicates(subset = None, keep = 'first')
print('''''')

print('Test data shape without duplication: ',test_data.shape)
print('Test data target distribution without duplication: ' , test_data.groupby('income').size())


Train data shape:  (32560, 15)
Train data target distribution:  income
<=50K    24719
>50K      7841
dtype: int64

Controlling duplication for train data:  (24, 15)

Train data shape without duplication:  (32536, 15)
Train data target distribution without duplication:  income
<=50K    24697
>50K      7839
dtype: int64
------------------------------------------
Test data shape:  (16281, 15)
Test data target distribution:  income
<=50K.    12435
>50K.      3846
dtype: int64

Controlling duplication for test data:  (5, 15)

Test data shape without duplication:  (16276, 15)
Test data target distribution without duplication:  income
<=50K.    12430
>50K.      3846
dtype: int64


In [220]:
print('Train data Info')
print(train_data.info())

print('''''')

print('Test data Info')
print(test_data.info())

Train data Info
<class 'pandas.core.frame.DataFrame'>
Int64Index: 32536 entries, 0 to 32559
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32536 non-null  int64 
 1   workClass       30700 non-null  object
 2   fnlwgt          32536 non-null  int64 
 3   education       32536 non-null  object
 4   education-num   32536 non-null  int64 
 5   marital-status  32536 non-null  object
 6   occupation      30693 non-null  object
 7   relationship    32536 non-null  object
 8   race            32536 non-null  object
 9   sex             32536 non-null  object
 10  capital-gain    32536 non-null  int64 
 11  capital-loss    32536 non-null  int64 
 12  hours-per-week  32536 non-null  int64 
 13  native-country  31954 non-null  object
 14  income          32536 non-null  object
dtypes: int64(6), object(9)
memory usage: 4.0+ MB
None

Test data Info
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16276 

**Summary on Train Data**

In [221]:
train_data.describe(include = 'all')

,age,workClass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
count,32536.000000,30700,3.253600e+04,32536,32536.000000,32536,30693,32536,32536,32536,32536.000000,32536.000000,32536.000000,31954,32536
unique,NaN,8,NaN,16,NaN,7,14,6,5,2,NaN,NaN,NaN,41,2
top,NaN,Private,NaN,HS-grad,NaN,Married-civ-spouse,Prof-specialty,Husband,White,Male,NaN,NaN,NaN,United-States,<=50K
freq,NaN,22673,NaN,10494,NaN,14970,4136,13187,27794,21774,NaN,NaN,NaN,29152,24697
mean,38.585536,NaN,1.897843e+05,NaN,10.081725,NaN,NaN,NaN,NaN,NaN,1078.410069,87.370912,40.440343,NaN,NaN
std,13.638193,NaN,1.055563e+05,NaN,2.571622,NaN,NaN,NaN,NaN,NaN,7388.068465,403.107737,12.347079,NaN,NaN
min,17.000000,NaN,1.228500e+04,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,NaN
25%,28.000000,NaN,1.178315e+05,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
50%,37.000000,NaN,1.783560e+05,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
75%,48.000000,NaN,2.369932e+05,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,45.000000,NaN,NaN


**Summary on Test Data**

In [222]:
test_data.describe(include = 'all')

,age,workClass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
count,16276.000000,15313,1.627600e+04,16276,16276.000000,16276,15310,16276,16276,16276,16276.000000,16276.000000,16276.000000,16002,16276
unique,NaN,8,NaN,16,NaN,7,14,6,5,2,NaN,NaN,NaN,40,2
top,NaN,Private,NaN,HS-grad,NaN,Married-civ-spouse,Prof-specialty,Husband,White,Male,NaN,NaN,NaN,United-States,<=50K.
freq,NaN,11206,NaN,5283,NaN,7402,2031,6522,13941,10856,NaN,NaN,NaN,14657,12430
mean,38.770890,NaN,1.894421e+05,NaN,10.072438,NaN,NaN,NaN,NaN,NaN,1082.237466,87.926272,40.394507,NaN,NaN
std,13.849484,NaN,1.057086e+05,NaN,2.567570,NaN,NaN,NaN,NaN,NaN,7585.077133,403.164257,12.478902,NaN,NaN
min,17.000000,NaN,1.349200e+04,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,NaN
25%,28.000000,NaN,1.167435e+05,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
50%,37.000000,NaN,1.778295e+05,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
75%,48.000000,NaN,2.383840e+05,NaN,12.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,45.000000,NaN,NaN


In [223]:
set(train_data["native-country"].unique())-set(test_data["native-country"].unique())

{'Holand-Netherlands'}

Native-country column has the category above in the train set and it is not in the test set. It is important to know it. In algorithms like Lightgbm, not-existing categories in the test set is handled without manual intervention. However in XgBoost, we need to make manual one-hot encoding since the algorithm does not do it itself. Since the test set has a missing category, we either need to make the encoding twice, for the train set and by creating a dummy column whose value is zero for all records for the test set, or we will do the encoding from the appended version of the two datasets. (Note, when using XgBoost in H2o even though we encode this column from the train set, it handles the unknown category itself in the test set. So we do not have to append data to make the encoding in H2o.) For the simplicity, we will do required operations on the appended data set, then will resplit the data into train and test sets.

**Example of Train Data**

In [224]:
train_data.head()

,age,workClass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


**Example of Test Data**

Values for income column have dots (.) at the end. However it does not have dots in the train set.

In [225]:
test_data.head()

,age,workClass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K.


**Looks Like Either Education or Education-num Column is Redundant**

In [226]:
edu_data=pd.DataFrame(train_data.groupby(['education', 'education-num']).size()).sort_values(by=['education-num']).reset_index()
edu_data.columns=["education","education-num","count"]
edu_data

,education,education-num,count
0,Preschool,1,50
1,1st-4th,2,166
2,5th-6th,3,332
3,7th-8th,4,645
4,9th,5,514
5,10th,6,933
6,11th,7,1175
7,12th,8,433
8,HS-grad,9,10494
9,Some-college,10,7282


**Append Two Sets**

In [227]:
data = train_data.append(test_data)
print('Appended data shape:', data.shape)

Appended data shape: (48812, 15)


**Define Function to Split**

In [228]:
def func_to_resplit(frame,algorithm):
    
    """Function to split appended data later, input and output are either in h2o frame or in pandas dataframe format
    """
       
    if algorithm=="H2O":
        X_train = frame[:train_data.shape[0],:frame.shape[1]-1]
        y_train = frame[:train_data.shape[0],-1]

        X_test = frame[train_data.shape[0]:,:frame.shape[1]-1]
        y_test = frame[train_data.shape[0]:,-1]

    else:
        shuffled_train_data = frame.sample(frac=1).reset_index(drop=True)
        X_train = shuffled_train_data.iloc[:train_data.shape[0],:frame.shape[1]-1]
        y_train = shuffled_train_data.iloc[:train_data.shape[0],-1]

        X_test = shuffled_train_data.iloc[train_data.shape[0]:,:frame.shape[1]-1]
        y_test = shuffled_train_data.iloc[train_data.shape[0]:,-1]
         
    print('X_train shape:', X_train.shape)
    print('y_train shape:', y_train.shape)
    print("-------------")
    print('X_test shape:', X_test.shape)
    print('y_test shape:', y_test.shape)
    
    return X_train,y_train,X_test,y_test
    

**Operations on Data for XgBoost**

In [229]:
if ALGORITHM=="XGBOOST":

    #One Hot Encoding of the Categorical features 
    cols_to_one_hot=["workClass","education","marital-status","occupation","relationship","race","sex","native-country"]
    one_hot_data=pd.get_dummies(data[cols_to_one_hot])
    
    #Merging one hot encoded features with our dataset 'data' 
    data = pd.concat([data,one_hot_data],axis=1)
 
    #Removing categorical features 
    data.drop(['workClass','education','marital-status','occupation','relationship','race','sex','native-country','education-num'],axis=1,inplace=True)

    #Here our target variable is 'Income' with values as 1 or 0.  
    data.loc[data['income'].isin(['<=50K','<=50K.']), 'income']=0
    data.loc[data['income'].isin(['>50K','>50K.']) ,'income']=1
    
    #Put income column to the end
    data["income_level"]=data.income
    data.drop(columns=["income"],inplace=True)

    #Resplit the Data into train and test sets
    X_train,y_train,X_test,y_test=func_to_resplit(data,ALGORITHM)


**Operations on Data for LightGBM**

In [230]:
if ALGORITHM=="LIGHTGBM":
    
    #Remove redundant column
    data.drop(columns = ['education-num'],inplace = True)
    
    #Here our target variable is 'Income' with values as 1 or 0.
    data.loc[data['income'].isin(['<=50K','<=50K.']), 'income']=0
    data.loc[data['income'].isin(['>50K','>50K.']) ,'income']=1
    
    #Set categorical features
    categorical_features = ['workClass','education','marital-status','occupation','relationship','race','sex','native-country']
    data[categorical_features]=data[categorical_features].astype('category')
    data['income']=data['income'].astype('category')
    
    #Resplit the Data into train and test sets
    X_train,y_train,X_test,y_test=func_to_resplit(data,ALGORITHM)

In [231]:
if ALGORITHM=="H2O":
    
    import h2o
    
    #Remove redundant column
    data.drop(columns = ['education-num'],inplace = True)
    
    #Correct target values in the test set
    data.loc[data['income'].isin(['<=50K.']), 'income']='<=50K'
    data.loc[data['income'].isin(['>50K.']) ,'income']='>50K'
    
   
    h2o.init()
    #Pandas to h20 frame
    hf_data = h2o.H2OFrame(data)
    
    #Set categorical features
    categorical_features = ['workClass','education','marital-status','occupation',
                            'relationship','race','sex','native-country','income']
    # Format columns
    for key in hf_data.types:
        if key in categorical_features:
            hf_data[key] = hf_data[key].asfactor()
        else:
            hf_data[key] = hf_data[key].asnumeric()
    
    #Resplit the Frame into train and test sets in h2o frame format
    X_train,y_train,X_test,y_test=func_to_resplit(hf_data,ALGORITHM)
      

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,65 days 21 hours 51 mins
H2O_cluster_timezone:,Europe/Istanbul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,2 months and 15 days
H2O_cluster_name:,H2O_from_python_oblomov_d6q8h6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.877 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
X_train shape: (32536, 13)
y_train shape: (32536, 1)
-------------
X_test shape: (16276, 13)
y_test shape: (16276, 1)


**Save Splitted Data For Modeling**

In [232]:
if ALGORITHM=="H2O":
    #Saves in h2o frame format
    h2o.download_csv(X_train,"./Model_Data/"+ALGORITHM+"_X_train.csv")
    h2o.download_csv(y_train,"./Model_Data/"+ALGORITHM+"_y_train.csv")
    h2o.download_csv(X_test,"./Model_Data/"+ALGORITHM+"_X_test.csv")
    h2o.download_csv(y_test,"./Model_Data/"+ALGORITHM+"_y_test.csv")
else:
    #Saves dataframe to csv
    X_train.to_csv("./Model_Data/"+ALGORITHM+"_X_train.csv")
    y_train.to_csv("./Model_Data/"+ALGORITHM+"_y_train.csv")
    X_test.to_csv("./Model_Data/"+ALGORITHM+"_X_test.csv")
    y_test.to_csv("./Model_Data/"+ALGORITHM+"_y_test.csv")

End of Data Preparation